In [115]:
import tejapi
import pandas as pd
import numpy as np
import copy
#繪圖
import plotly.graph_objects as go
from plotly import subplots
import plotly.express as px
tejapi.ApiConfig.api_key = 'Your Key'
tejapi.ApiConfig.ignoretz = True

In [116]:
#股價
stock_data = tejapi.get('TWN/APRCD',
                  coid= '2002',
                  mdate={'gte': '2021-01-01'}, 
                  opts={'columns': ['coid', 'mdate', 'open_d', 'close_d']},
                  chinese_column_name=True,
                  paginate=True)
stock_data

,證券代碼,年月日,開盤價(元),收盤價(元)
None,,,,
0,2002,2021-01-04,24.90,24.95
1,2002,2021-01-05,25.15,26.00
2,2002,2021-01-06,26.50,25.50
3,2002,2021-01-07,25.70,25.70
4,2002,2021-01-08,25.95,26.00
...,...,...,...,...
188,2002,2021-10-14,32.65,32.75
189,2002,2021-10-15,33.20,33.20
190,2002,2021-10-18,33.45,33.65


In [117]:
#策略一: KD指標
kd_data = copy.deepcopy(stock_data)
#設9天
kd_data['9_high'] = kd_data['收盤價(元)'].rolling(9).max()
kd_data['9_low'] = kd_data['收盤價(元)'].rolling(9).min()
#rsv 缺值以50填補
kd_data['rsv'] = ((kd_data['收盤價(元)']-kd_data['9_low'])/(kd_data['9_high']-kd_data['9_low']))*100
kd_data['rsv'] =  kd_data['rsv'].fillna(50)
kd_data

,證券代碼,年月日,開盤價(元),收盤價(元),9_high,9_low,rsv
None,,,,,,,
0,2002,2021-01-04,24.90,24.95,NaN,NaN,50.000000
1,2002,2021-01-05,25.15,26.00,NaN,NaN,50.000000
2,2002,2021-01-06,26.50,25.50,NaN,NaN,50.000000
3,2002,2021-01-07,25.70,25.70,NaN,NaN,50.000000
4,2002,2021-01-08,25.95,26.00,NaN,NaN,50.000000
...,...,...,...,...,...,...,...
188,2002,2021-10-14,32.65,32.75,35.85,32.4,10.144928
189,2002,2021-10-15,33.20,33.20,35.60,32.4,25.000000
190,2002,2021-10-18,33.45,33.65,35.60,32.4,39.062500


In [118]:
#初始化kd 
kd_data['k'] = 0
kd_data['d'] = 0
kd_data['買賣股數'] = 0

#迴圈修正k d值  並填上訊號 每次操作為1000股，不連續進行買入or賣出
hold = 0
for i in range(len(kd_data)):
    #先算出當期的k d 值
    if i == 0:
        kd_data.loc[i,'k'] = 50
        kd_data.loc[i,'d'] = 50
    else:
        kd_data.loc[i,'k'] = (2/3* kd_data.loc[i-1,'k']) + (kd_data.loc[i,'rsv']/3)
        kd_data.loc[i,'d'] = (2/3* kd_data.loc[i-1,'d']) + (kd_data.loc[i,'k']/3)
    
    if kd_data.loc[i, 'k'] <= 20:
        if hold == 0:                                   #無部位 隔天開盤才買進 
            kd_data.loc[i+1, '買賣股數'] = 1000        
            hold = 1                                    #hold=1代表手中有持股 0代表無持股
            
    elif kd_data.loc[i,'k'] >= 80:
        if hold > 0:                                    #有long部位  隔天才進行賣出
            kd_data.loc[i+1,'買賣股數'] = -1000     
            hold = 0

In [119]:
kd_data

,證券代碼,年月日,開盤價(元),收盤價(元),9_high,9_low,rsv,k,d,買賣股數
None,,,,,,,,,,
0,2002,2021-01-04,24.90,24.95,NaN,NaN,50.000000,50.000000,50.000000,0
1,2002,2021-01-05,25.15,26.00,NaN,NaN,50.000000,50.000000,50.000000,0
2,2002,2021-01-06,26.50,25.50,NaN,NaN,50.000000,50.000000,50.000000,0
3,2002,2021-01-07,25.70,25.70,NaN,NaN,50.000000,50.000000,50.000000,0
4,2002,2021-01-08,25.95,26.00,NaN,NaN,50.000000,50.000000,50.000000,0
...,...,...,...,...,...,...,...,...,...,...
188,2002,2021-10-14,32.65,32.75,35.85,32.4,10.144928,6.543245,7.998023,0
189,2002,2021-10-15,33.20,33.20,35.60,32.4,25.000000,12.695496,9.563848,0
190,2002,2021-10-18,33.45,33.65,35.60,32.4,39.062500,21.484498,13.537398,0


In [120]:
#買賣點視覺化

graph_price = go.Scatter(x = kd_data['年月日'],
                         y = kd_data['收盤價(元)'],                  
                         mode = 'lines', line_color = 'grey', 
                         name='股價')

graph_buy= go.Scatter(x=kd_data["年月日"], y= np.where(kd_data['買賣股數'] > 0, kd_data['收盤價(元)'], np.nan), 
                            mode='markers', marker_symbol="triangle-up", marker_color="red", marker_size=15,
                            name='Buy', text = "買入: "+ kd_data['買賣股數'].apply(str) + '股')
        
graph_sell= go.Scatter(x=kd_data["年月日"], y= np.where(kd_data['買賣股數'] < 0, kd_data['收盤價(元)'], np.nan), 
                            mode='markers', marker_symbol="triangle-down", marker_color="limegreen", marker_size=15,
                            name='Sell', text = "賣出: "+ abs(kd_data['買賣股數']).apply(str) + '股')


#資料
fig = go.Figure()
fig.add_trace(graph_price)
fig.add_trace(graph_buy)
fig.add_trace(graph_sell)

#x y軸設定
fig.update_layout(title='買賣點觀察',
                 xaxis_title="日期",
                 yaxis_title="股價",)
fig.show()

In [121]:
def ret(data, principal):
    
    #計算成本
    data['手續費'] = data['開盤價(元)']* abs(data['買賣股數'])*0.001425
    data['手續費'] = np.where((data['手續費']>0)&(data['手續費'] <20), 20, data['手續費'])
    data['證交稅'] = np.where(data['買賣股數']<0, data['開盤價(元)']* abs(data['買賣股數'])*0.003, 0)
    data['摩擦成本'] = (data['手續費'] + data['證交稅']).apply(np.floor)
    
    #計算資產價值
    data['股票價值'] = data['買賣股數'].cumsum() * data['收盤價(元)']
    data['現金價值'] = principal - data['摩擦成本'] + (data['開盤價(元)']* -data['買賣股數']).cumsum() 
    data['資產價值'] = data['股票價值'] + data['現金價值']
    
    #計算報酬率
    data['當日價值變動(%)'] = (data['資產價值']/data['資產價值'].shift(1) - 1)*100
    data['累計報酬(%)'] = (data['資產價值']/principal - 1)*100
    
    return data

In [122]:
#起初現金部位為30000
kd_return = ret(data = kd_data, principal = 30000)
kd_return

,證券代碼,年月日,開盤價(元),收盤價(元),9_high,9_low,rsv,k,d,買賣股數,手續費,證交稅,摩擦成本,股票價值,現金價值,資產價值,當日價值變動(%),累計報酬(%)
None,,,,,,,,,,,,,,,,,,
0,2002,2021-01-04,24.90,24.95,NaN,NaN,50.000000,50.000000,50.000000,0,0.0,0.0,0.0,0.0,30000.0,30000.0,NaN,0.000000
1,2002,2021-01-05,25.15,26.00,NaN,NaN,50.000000,50.000000,50.000000,0,0.0,0.0,0.0,0.0,30000.0,30000.0,0.000000,0.000000
2,2002,2021-01-06,26.50,25.50,NaN,NaN,50.000000,50.000000,50.000000,0,0.0,0.0,0.0,0.0,30000.0,30000.0,0.000000,0.000000
3,2002,2021-01-07,25.70,25.70,NaN,NaN,50.000000,50.000000,50.000000,0,0.0,0.0,0.0,0.0,30000.0,30000.0,0.000000,0.000000
4,2002,2021-01-08,25.95,26.00,NaN,NaN,50.000000,50.000000,50.000000,0,0.0,0.0,0.0,0.0,30000.0,30000.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2002,2021-10-14,32.65,32.75,35.85,32.4,10.144928,6.543245,7.998023,0,0.0,0.0,0.0,32750.0,5900.0,38650.0,0.913838,28.833333
189,2002,2021-10-15,33.20,33.20,35.60,32.4,25.000000,12.695496,9.563848,0,0.0,0.0,0.0,33200.0,5900.0,39100.0,1.164295,30.333333
190,2002,2021-10-18,33.45,33.65,35.60,32.4,39.062500,21.484498,13.537398,0,0.0,0.0,0.0,33650.0,5900.0,39550.0,1.150895,31.833333


In [123]:
#策略二: 買入並持有報酬率
bh_data = copy.deepcopy(stock_data)
bh_data

,證券代碼,年月日,開盤價(元),收盤價(元)
None,,,,
0,2002,2021-01-04,24.90,24.95
1,2002,2021-01-05,25.15,26.00
2,2002,2021-01-06,26.50,25.50
3,2002,2021-01-07,25.70,25.70
4,2002,2021-01-08,25.95,26.00
...,...,...,...,...
188,2002,2021-10-14,32.65,32.75
189,2002,2021-10-15,33.20,33.20
190,2002,2021-10-18,33.45,33.65


In [124]:
bh_data['買賣股數'] = 0
bh_data.loc[0, '買賣股數'] = 1000
bh_data.loc[len(bh_data)-1, '買賣股數'] = -1000
bh_data

,證券代碼,年月日,開盤價(元),收盤價(元),買賣股數
None,,,,,
0,2002,2021-01-04,24.90,24.95,1000
1,2002,2021-01-05,25.15,26.00,0
2,2002,2021-01-06,26.50,25.50,0
3,2002,2021-01-07,25.70,25.70,0
4,2002,2021-01-08,25.95,26.00,0
...,...,...,...,...,...
188,2002,2021-10-14,32.65,32.75,0
189,2002,2021-10-15,33.20,33.20,0
190,2002,2021-10-18,33.45,33.65,0


In [125]:
bh_return = ret(data=bh_data, principal = 30000)
bh_return

,證券代碼,年月日,開盤價(元),收盤價(元),買賣股數,手續費,證交稅,摩擦成本,股票價值,現金價值,資產價值,當日價值變動(%),累計報酬(%)
None,,,,,,,,,,,,,
0,2002,2021-01-04,24.90,24.95,1000,35.48250,0.00,35.0,24950.0,5065.0,30015.0,NaN,0.050000
1,2002,2021-01-05,25.15,26.00,0,0.00000,0.00,0.0,26000.0,5100.0,31100.0,3.614859,3.666667
2,2002,2021-01-06,26.50,25.50,0,0.00000,0.00,0.0,25500.0,5100.0,30600.0,-1.607717,2.000000
3,2002,2021-01-07,25.70,25.70,0,0.00000,0.00,0.0,25700.0,5100.0,30800.0,0.653595,2.666667
4,2002,2021-01-08,25.95,26.00,0,0.00000,0.00,0.0,26000.0,5100.0,31100.0,0.974026,3.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2002,2021-10-14,32.65,32.75,0,0.00000,0.00,0.0,32750.0,5100.0,37850.0,0.933333,26.166667
189,2002,2021-10-15,33.20,33.20,0,0.00000,0.00,0.0,33200.0,5100.0,38300.0,1.188904,27.666667
190,2002,2021-10-18,33.45,33.65,0,0.00000,0.00,0.0,33650.0,5100.0,38750.0,1.174935,29.166667


In [127]:
market = tejapi.get('TWN/APRCD',
                    coid = 'Y9997',
                    mdate = {'gte':'2021-01-01'},
                    opts = {'columns':['coid','mdate', 'roi']},
                    chinese_column_name = True)
market

,證券代碼,年月日,報酬率％
None,,,
0,Y9997,2021-01-04,1.1505
1,Y9997,2021-01-05,0.6576
2,Y9997,2021-01-06,-0.1126
3,Y9997,2021-01-07,1.5408
4,Y9997,2021-01-08,1.6429
...,...,...,...
188,Y9997,2021-10-14,0.2420
189,Y9997,2021-10-15,2.4037
190,Y9997,2021-10-18,-0.4511


In [128]:
market['累計報酬(%)'] = (market['報酬率％'].apply(lambda x:0.01*x +1).cumprod() - 1)*100
market

,證券代碼,年月日,報酬率％,累計報酬(%)
None,,,,
0,Y9997,2021-01-04,1.1505,1.150500
1,Y9997,2021-01-05,0.6576,1.815666
2,Y9997,2021-01-06,-0.1126,1.701021
3,Y9997,2021-01-07,1.5408,3.268031
4,Y9997,2021-01-08,1.6429,4.964621
...,...,...,...,...
188,Y9997,2021-10-14,0.2420,14.130538
189,Y9997,2021-10-15,2.4037,16.873894
190,Y9997,2021-10-18,-0.4511,16.346675


In [129]:
fig = px.line()
fig.add_scatter(x = kd_return['年月日'], y = kd_return['累計報酬(%)'], name = "KD策略")
fig.add_scatter(x = bh_return['年月日'], y = bh_return['累計報酬(%)'], name = "買入持有策略")
fig.add_scatter(x = market['年月日'], y = market['累計報酬(%)'], name = "市場表現")

fig.update_layout(title='策略累計報酬比較',
                 xaxis_title="日期",
                 yaxis_title="累計報酬(%)",)
fig.show()

In [130]:
cagr = [(((kd_return['累計報酬(%)'].values[-1]*0.01 + 1)**(252/len(kd_return))) - 1)*100, ((bh_return['累計報酬(%)'].values[-1]*0.01 + 1)**(252/len(bh_return)) - 1)*100, 100*((market['累計報酬(%)'].values[-1]*0.01 + 1)**(252/len(market)) - 1)]
cagr

[42.98197572241966, 40.38474378427066, 23.600090276029427]

In [131]:
std = [kd_return['當日價值變動(%)'].std()*(252**0.5), bh_return['當日價值變動(%)'].std()*(252**0.5), market['報酬率％'].std()*(252**0.5)]
std

[24.487652761659636, 40.775155682215804, 18.664059421460216]

In [132]:
#無風險利率1%
sharpe_ratio = [(cagr[0] - 1)/std[0],(cagr[1] - 1)/std[1],(cagr[2] - 1)/std[2]]
sharpe_ratio

[1.7144140408651547, 0.9659005128323379, 1.2108882513545527]

In [133]:
performance = pd.DataFrame([cagr,std,sharpe_ratio], columns = ['KD策略','買入持有','市場'], index = ['年化報酬(%)','年化標準差(%)','夏普比率'])
performance

,KD策略,買入持有,市場
年化報酬(%),42.981976,40.384744,23.600090
年化標準差(%),24.487653,40.775156,18.664059
夏普比率,1.714414,0.965901,1.210888
